CPU + RAM 60-80 GB (เลือกใช้ GPU: A100 ของ Colab Pro+)

In [1]:
! mkdir -p ./corpus_input

In [2]:
! wget -O ./corpus_input/1.pdf https://storage.googleapis.com/corpus_input/1.pdf
! wget -O ./corpus_input/2.pdf https://storage.googleapis.com/corpus_input/2.pdf
! wget -O ./corpus_input/3.pdf https://storage.googleapis.com/corpus_input/3.pdf
! wget -O ./corpus_input/44.pdf https://storage.googleapis.com/corpus_input/44.pdf
! wget -O ./corpus_input/5555.pdf https://storage.googleapis.com/corpus_input/5555.pdf

--2024-06-07 14:52:01--  https://storage.googleapis.com/corpus_input/1.pdf
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.207, 142.251.10.207, 142.251.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2477718 (2.4M) [application/pdf]
Saving to: ‘./corpus_input/1.pdf’

./corpus_input/1.pd 100%[===================>]   2.36M  1.41MB/s    in 1.7s    

2024-06-07 14:52:04 (1.41 MB/s) - ‘./corpus_input/1.pdf’ saved [2477718/2477718]

--2024-06-07 14:52:04--  https://storage.googleapis.com/corpus_input/2.pdf
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.207, 142.251.10.207, 142.251.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2818771 (2.7M) [application/pdf]
Saving to: ‘./corpus_input/2.pdf’

./corpus_input/2.pd 100%[==

In [3]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [4]:
question_text = "ผมมีไข้และมีผื่นขึ้นตามตัว ผมเป็นโรคอะไร?"

In [5]:
device = 'cpu'

# 1. Text Response Generators - Model loading and Tokenization

In [6]:
import torch
from accelerate import Accelerator

accelerator = Accelerator()

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Aekanun/openthaigpt-MedChatModel")
model = AutoModelForCausalLM.from_pretrained("Aekanun/openthaigpt-MedChatModel")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/718k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.27G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/419M [00:00<?, ?B/s]

In [8]:
def generate(question: str, context: str):
    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""
    chat = [{"role": "user", "content": prompt}]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=250,
            do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response[len(formatted_prompt) :]  # remove input prompt from reponse
    response = response.replace("<eos>", "")  # remove eos token
    return response

In [9]:
print(generate(question=question_text, context=""))

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


อาการไข้และผื่นตามตัวอาจบ่งบอกถึงโรคหัดหรือโรคหัดเยอรมัน โรคหัดเป็นโรคติดเชื้อไวรัสที่แพร่กระจายผ่านละอองฝอยเมื่อไอ จาม หรือสัมผัสใกล้ชิดกับผู้ติดเชื้อ อาการทั่วไปของหัด ได้แก่ มีไข้สูง ปวดศีรษะ ปวดเมื่อยตามร่างกาย มีผื่นแดงเป็นตุ่มน้ำใสที่ผิวหนังและเยื่อบุตา และอาการอื่นๆ เช่น เจ็บคอ ไอ และน้ำมูกไหล โรคหัดเยอรมันเป็นโรคหัดที่รุนแรงกว่าและมักมีอาการรุนแรงกว่า เช่น มีไข้สูง ปวดศีรษะ ปวดเมื่อยตามร่างกาย มีผื่นแดงเป็นตุ่มน้ำใสที่ผิวหนังและเยื่อบุตา และอาการอื่นๆ เช่น เจ็บคอ ไอ และน้ำมูกไหล หากคุณมีอาการเหล่านี้ คุณควรไปพบแพทย์เพื่อรับการวินิจฉัยและการรักษาที่เหมาะสม</s>


# 2. Splitting Documents (PDF)

In [10]:
! pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [11]:
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.3 MB/s eta 0:00:00


In [12]:
!pip install pypdf tiktoken langchain sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.4 MB/s eta 0:00:00


In [13]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load PDFs
loaders = [
    #PyPDFLoader("/home/jupyter/corpus_input/pobpad-1.pdf"),
    PyPDFLoader("./corpus_input/1.pdf"),
    PyPDFLoader("./corpus_input/2.pdf"),
    PyPDFLoader("./corpus_input/3.pdf"),
    PyPDFLoader("./corpus_input/44.pdf"),
    PyPDFLoader("./corpus_input/5555.pdf"),

]
pages = []
for loader in loaders:
    pages.extend(loader.load())

# split text to chunks
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=AutoTokenizer.from_pretrained(
        "sentence-transformers/all-MiniLM-L12-v2"
     ),
     chunk_size=256,
     chunk_overlap=32,
     strip_whitespace=True,
)

docs = text_splitter.split_documents(pages)

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# 3. Embedding and Creating Vector Database

In [14]:
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)

encoder = HuggingFaceEmbeddings(
    model_name = 'sentence-transformers/all-MiniLM-L12-v2',
    model_kwargs = {'device': device}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.9 MB/s eta 0:00:00


In [16]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

faiss_db = FAISS.from_documents(
    docs, encoder, distance_strategy=DistanceStrategy.COSINE
)

# 4. Document Retrieval

In [17]:
retrieved_docs = faiss_db.similarity_search(question_text, k=1)

In [18]:
retrieved_docs

[Document(page_content='ผ้าเช็ดตัว หนังสือ โทรศัพท์ ของเล่นเป็นต้น) หรือสิ่งแวดล้อม เมื่อคนปกติมาสัมผัสถูกมือของผู้ป่วย สิ่งของเครื่องใช้ หรือสิ่งแวดล้อมที่แปดเปื้อนเชื้อ เชื้อก็จะติดมากับมือของคนๆ นั้น เมื่อใช้นิ้วมือขยี้ตาหรือแคะไชจมูกเชื้อหัดเยอรมันก็จะเข้าสู่ร่างกายได้ (การสัมผัสผื่นที่ผิวหนังของผู้ป่วยไม่ได้ติดโรคได้ และในผู้ที่มีภาวะภูมิคุ้มกันต้านทานโรคบกพร่องจะไม่เป็นการเพิ่มความเสี่ยงต่อการติดเชื้อหัดเยอรมันง่ายกว่าผู้ที่มีภูมิคุ้มกันปกติแต่อย่างใด ซึ่งต่างจากโรคหัดที่ผู้ที่มีภาวะภูมิคุ้มต้านทานโรคบกพร่องจะมีโอกาสติดโรคหัดได้ง่ายหัดเยอรมัน อาการ สาเหตุ และการรักษาโรคหัดเยอรมัน 10 วิธี !!about:reader?url=https%3A%2F%2Fmedthai.com%2F%25E0%25B9...\n6 of 5315/4/2567 BE, 19:19', metadata={'source': './corpus_input/1.pdf', 'page': 5})]

In [19]:
faiss_db